In [1]:
import numpy as np
import pandas as pd
from rapidfuzz import process

In [2]:
# helper function: turn np.nan into ""
def deNaN(series):
    return series.apply(lambda x: "" if type(x) != str else x)

In [3]:
# helper function to combine columns with data into one column with all unique values
# we use this to combine name, town, state and person columns into just one column
def combineCols(df, state, num=3, namenum=3):
    change_df_agg = pd.DataFrame(columns=['old', 'new', 'type'])
    for col in ['town', 'state', 'occupation']:
        if num == 3:
            df[col] = [set([" ".join(x.split()) for x in [t1, t2, t3] if not pd.isnull(x)]) for t1, t2, t3 in
                       zip(df[col + '1'],
                           df[col + '2'],
                           df[col + '3'])]
        else:
            df[col] = [set([" ".join(x.split()) for x in [t1, t2] if not pd.isnull(x)]) for t1, t2 in zip(df[col + '1'],
                                                                                                          df[
                                                                                                              col + '2'])]
        if not any(df[col].apply(lambda x: len(x) > 1).tolist()):
            # change set to string
            print("reformatting {}".format(col))
            df[col] = df[col].apply(lambda x: x.pop() if x != set() else np.nan)
        else:
            print("{} column has multiple unique entries".format(col))
            print("see table at end for new entries")
            # keep the value that has the most characters, otherwise change set to string
            old = df[df[col].apply(lambda x: len(x) > 1)][col]
            df[col] = df[col].apply(
                lambda x: x.pop() if len(x) == 1 else np.nan if x == set() else max(list(x), key=len))
            # new dataframe to keep track of changes
            # create copy of change_df that removes all duplicates by turning old column, which is
            # of type set to type string
            change_df = pd.DataFrame([old, df.loc[old.index][col]]).T
            change_df.columns = ['old', 'new']
            change_df['type'] = col
            change_df_str = change_df.copy()
            change_df_str['old'] = change_df_str['old'].astype(str)
            change_df_str = change_df_str.drop_duplicates()
            # add filtered database of changes to aggregate dataset
            change_df_agg = pd.concat([change_df_agg, change_df.loc[change_df_str.index]])

    # add functions to combine asset totals, handle missing debt values
    if namenum == 3:
        df['Name 1'] = deNaN(df['First Name']) + " " + deNaN(df['Last Name'])
        df['Name 2'] = deNaN(df['First Name.1']) + " " + deNaN(df['Last Name.1'])
        df['Name 3'] = deNaN(df['First Name.2']) + " " + deNaN(df['Last Name.2'])
        df['Name'] = list(
            list(set([x.replace("  ", " ").strip() for x in [name1, name2, name3] if x.strip() != ""])) for
            name1, name2, name3 in zip(df['Name 1'], df['Name 2'], df['Name 3']))
    df.loc[df.query('state.isna()').index, 'state'] = state
    df['state_data'] = state
    df['state_data_index'] = np.arange(1, df.shape[0] + 1, 1)

    return df[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
               'town', 'state', 'occupation', 'Name', 'state_data', 'state_data_index']], change_df_agg

In [4]:
def cleanSingleColumn(df, state):
    # convert name column types
    df['Name'] = deNaN(df['First Name']) + " " + deNaN(df['Last Name'])
    df['Name'] = df['Name'].apply(lambda x: [x])
    # impute state
    df.loc[df.query('state.isna()').index, 'state'] = state
    df['state_data'] = state
    df['state_data_index'] = np.arange(1, df.shape[0] + 1, 1)
    return df

## Import Raw Data

### Connecticut

In [5]:
CD_all = pd.DataFrame(columns=['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                               'town', 'state', 'occupation', 'Name', 'state_data', 'state_data_index'])
ASD_all = pd.DataFrame(columns=['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                                'town', 'state', 'occupation', 'Name', 'state_data', 'state_data_index'])

In [6]:
# importing desired columns and rename
CT_CD_raw = pd.read_excel("../data_raw/post1790/CT/CT_post1790_CD_ledger.xlsx",
                          header=13, usecols='H, I, J, K, L, N, O, X, Y, Z, AA, AB, AD, AE, AN, AO, AP, AQ, AR, AT, AU')
CT_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
CT_CD, change_df = combineCols(CT_CD_raw, 'CT')
CD_all = pd.concat([CD_all, CT_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
reformatting occupation


,old,new,type
526,"{Miford, Milford}",Milford,town
799,"{Nowich, Norwich}",Norwich,town


In [7]:
# importing desired columns and rename
CT_ASD_raw = pd.read_excel("../data_raw/post1790/CT/CT_post1790_ASD_ledger.xlsx",
                           header=13,
                           usecols='H, I, J, K, L, N, O, X, Y, Z, AA, AB, AD, AE, AN, AO, AP, AQ, AR, AT, AU')
CT_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                      'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                      'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
CT_ASD, change_df = combineCols(CT_ASD_raw, 'CT')
ASD_all = pd.concat([ASD_all, CT_ASD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
125,"{Norwall, Norwalk}",Norwall,town
811,"{1st Society in Lyme, Treasuer 1st Society in ...",Treasuer 1st Society in Lyme,occupation


### Georgia
No need to do additional cleaning because there's only one state/city/occupation column

In [8]:
# importing desired columns and rename
# prepare loan dataset
GA_CD_raw = pd.read_excel("../data_raw/post1790/GA/T694_GA_Loan_Office_CD.xlsx",
                          header=10, usecols='Q, R, S, T, U, Z, AA, AB, AC, AD, AE')
# end of table
GA_CD_raw.drop([65, 66], inplace=True)

GA_CD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                     '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
GA_CD_raw = cleanSingleColumn(GA_CD_raw, 'GA')
GA_CD = GA_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation', 'Name']]

CD_all = pd.concat([CD_all, GA_CD])

### Maryland

In [9]:
#prepare loan dataset
MD_CD_raw = pd.read_excel("../data_raw/post1790/MD/MD_post1790_CD.xlsx",
                          header=11, usecols='G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
MD_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
MD_CD, change_df = combineCols(MD_CD_raw, 'MD')
CD_all = pd.concat([CD_all, MD_CD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


In [10]:
# prepare loan dataset
MD_ASD_raw = pd.read_excel("../data_raw/post1790/MD/MD_post1790_ASD.xlsx",
                           header=11, usecols='G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
MD_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                      'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                      'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
MD_ASD, change_df = combineCols(MD_ASD_raw, 'MD')
ASD_all = pd.concat([ASD_all, MD_ASD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


### North Carolina

In [11]:
NC_CD_raw = pd.read_excel("../data_raw/post1790/NC/T695_R4_NC_CD.xlsx",
                          header=11, usecols='J, K, L, M, N, W, X, Z, AA, AC, AD ')
NC_CD_raw.drop([74, 75], inplace=True)
NC_CD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                     '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']

NC_CD_raw = cleanSingleColumn(NC_CD_raw, 'NC')
NC_CD = NC_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation', 'Name']]
CD_all = pd.concat([CD_all, NC_CD])

In [12]:
NC_ASD_raw = pd.read_excel("../data_raw/post1790/NC/T695_R3_NC_ASD.xlsx", header=10,
                           usecols='H, I, J, K, L, P, Q, R, S, T, U')
NC_ASD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                      '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NC_ASD_raw = cleanSingleColumn(NC_ASD_raw, 'NC')
NC_ASD = NC_ASD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                     'town', 'state', 'occupation', 'Name']]
ASD_all = pd.concat([ASD_all, NC_ASD])

### New Hampshire

In [13]:
#prepare loan dataset
NH_CD_raw = pd.read_excel("../data_raw/post1790/NH/T652_R6_New_Hampshire_CD.xlsx",
                          header=10, usecols='I, J, K, L, M, N, O, P, Q, R, S').drop([219, 220])
NH_CD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                     '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NH_CD_raw = cleanSingleColumn(NH_CD_raw, 'NH')
NH_CD = NH_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation', 'Name']]

CD_all = pd.concat([CD_all, NH_CD])

In [14]:
NH_ASD_raw = pd.read_excel("../data_raw/post1790/NH/T652_New_Hampshire_ASD.xlsx", header=12,
                           usecols='G, H, I, J, K, M, N, V, W, X, Y, Z, AA, AB, AK, AL, AM, AN')
NH_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                      'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                      'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
# unify occupation, town and state columns
NH_ASD, change_df = combineCols(NH_ASD_raw, 'NH', 2)
ASD_all = pd.concat([ASD_all, NH_ASD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


### New York
Doesn't have town/occupation/state

### Pennsylvania

In [15]:
#prepare loan dataset
PA_CD_raw = pd.read_excel("../data_raw/post1790/PA/PA_post1790_CD.xlsx",
                          header=11, usecols='G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AJ, AK, AL, AM, AN, AO, AP')
PA_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
PA_CD, change_df = combineCols(PA_CD_raw, 'PA')
CD_all = pd.concat([CD_all, PA_CD])
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(change_df)

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
14,"{Carlisle Pennsylvania, Carlisle}",Carlisle Pennsylvania,town
39,"{Conecticutt, Connecticut}",Conecticutt,town
40,"{New Castle Delaware, Newcastle Delaware}",New Castle Delaware,town
113,"{Philadelphia, Philadelphia County}",Philadelphia County,town
131,"{State of Delawere, State of Delaware}",State of Delawere,town
210,"{Connecticutt, Connecticut}",Connecticutt,town
221,"{Bucks County, Bucks County Pennsylvania}",Bucks County Pennsylvania,town
267,"{Reading Berks County, Reading}",Reading Berks County,town
276,"{Reading Berks County, Reading Pennsylvania}",Reading Berks County,town
293,"{Northumberland, Northumb Country}",Northumb Country,town


### Rhode Island

In [16]:
#prepare loan dataset
RI_CD_raw = pd.read_excel("../data_raw/post1790/RI/T653_Rhode_Island_CD.xlsx",
                          header=11, usecols='G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
RI_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
RI_CD, change_df = combineCols(RI_CD_raw, 'RI')
CD_all = pd.concat([CD_all, RI_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
reformatting occupation


,old,new,type
26,"{Glocester, Gloucester}",Gloucester,town
371,"{Charlestown, Chalestown}",Charlestown,town
435,"{North Kingston, North Kingstone}",North Kingstone,town
462,"{North Kingston, North Kingstown}",North Kingstown,town
499,"{Smithfield, Smithfeild}",Smithfield,town


In [17]:
#prepare loan dataset
RI_ASD_raw = pd.read_excel("../data_raw/post1790/RI/T653_Rhode_Island_ASD.xlsx",
                           header=11,
                           usecols='H, I, J, K, L, N, O, X, Y, Z, AA, AB, AD, AE, AN, AO, AP, AQ, AR, AT, AU')
RI_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                      'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                      'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
RI_ASD, change_df = combineCols(RI_ASD_raw, 'RI')
ASD_all = pd.concat([CD_all, RI_ASD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


### South Carolina

In [18]:
#prepare loan dataset
SC_CD_raw = pd.read_excel("../data_raw/post1790/SC/Post_1790_South_Carolina_CD.xlsx",
                          header=11, usecols='D, E, F, G, H, M, N, S, T, U, V, W, AB, AC, AH, AI, AJ, AK, AL, AQ, AR')
SC_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
SC_CD, change_df = combineCols(SC_CD_raw, 'SC')
CD_all = pd.concat([CD_all, SC_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
50,"{Camden, Camden Planter}",Camden Planter,town
94,"{Peedee, Pee Dee}",Pee Dee,town
270,"{Long Cane, Long Cames}",Long Cames,town
292,"{Charlestom, Charleston}",Charlestom,town
45,"{as Executor to Henry Coram, as Executor to Jo...",as Executor to John Couturier,occupation
50,"{Planter as Executor Ely Kershaw, as Executor ...",Planter as Executor Ely Kershaw,occupation
182,"{Executor Philip Hawkins, Philip Hawkins}",Executor Philip Hawkins,occupation
256,"{as Guardian to Mary Deborah L. Gowdey, Guardi...",as Guardian to Mary Deborah L. Gowdey,occupation
305,"{Charleston Merchants, Merchants}",Charleston Merchants,occupation
383,"{Physician, Assignee of James Simons}",Assignee of James Simons,occupation


In [19]:
#prepare loan dataset
SC_ASD_raw = pd.read_excel("../data_raw/post1790/SC/Post_1790_South_Carolina_ASD_transfers_removed.xlsx", header=11,
                           usecols='D, E, F, G, H, M, N, O')
SC_ASD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_def_Dollar',
                      '3p_Dollar']
for col in ['6p_', '6p_def_', '3p_']:
    SC_ASD_raw[col + 'Cents'] = SC_ASD_raw[col + 'Dollar'] - np.round(SC_ASD_raw[col + 'Dollar'], 0)
    SC_ASD_raw[col + 'Dollar'] = np.round(SC_ASD_raw['6p_Dollar'], 0)

SC_ASD_raw = cleanSingleColumn(SC_ASD_raw, 'SC')
SC_ASD = SC_ASD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                     'town', 'state', 'occupation', 'Name']]
ASD_all = pd.concat([ASD_all, SC_ASD])

### Virginia

In [20]:
"""
VA CD not possible atm
"""

'\nVA CD not possible atm\n'

In [21]:
#prepare loan dataset
VA_ASD_raw = pd.read_excel("../data_raw/post1790/VA/VA_ASD.xlsx", header=11,
                           usecols='D, E, F, G, N, O, U, V, W, X, AE, AF, AL, AM, AN, AO, AW, AX')
VA_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'occupation1', '6p_Dollar', '6p_Cents',
                      'First Name.1', 'Last Name.1', 'town2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                      'First Name.2', 'Last Name.2', 'town3', 'occupation3', '3p_Dollar', '3p_Cents']
VA_ASD_raw['state1'] = np.nan
VA_ASD_raw['state2'] = np.nan
VA_ASD_raw['state3'] = np.nan
VA_ASD, change_df = combineCols(VA_ASD_raw, 'VA')
ASD_all = pd.concat([ASD_all, VA_ASD])
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(change_df)

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
0,"{The Town Petersburg, Town of Petersburg}",The Town Petersburg,town
1,"{City of Richmond, Richmond}",City of Richmond,town
2,"{Dinwiddie County, Dinwiddie Country}",Dinwiddie Country,town
3,"{Town of Petersbung, The Town Petersburg, Town...",The Town Petersburg,town
4,"{Halifox County, Halifax County}",Halifox County,town
5,"{Petersbung, Town of Petersburg}",Town of Petersburg,town
6,"{The City of Richmond, City of Richmond, Richm...",The City of Richmond,town
8,"{Rockbridge Country, Rockbridge County}",Rockbridge Country,town
9,"{Buckingham Country, Buckingham County}",Buckingham Country,town
10,"{Rockbridge County, Rocksbridge County}",Rocksbridge County,town


## Map each "town" to a county

In [22]:
# fuzzy string matching function
def fuzzyMatch(unmatched_towns, towns, crosswalk, primary_dict, dict_matchcol='primary_city', initial=True,
               score_threshold=85):
    if initial:
        print("\nFuzzy City name - county matches\n")
    else:
        print("\nFuzzy City name - county matches with string changes\n")
    printedtowns = []
    for town in unmatched_towns:
        # extract best match
        match_tuple = process.extractOne(town, [x for x in crosswalk[dict_matchcol] if not pd.isnull(x)])
        score = match_tuple[1]
        match = match_tuple[0]
        # if match above threshold, change + print match so we can hand check
        if score >= score_threshold:
            if dict_matchcol == 'primary_city':
                county = primary_dict[match]
            if dict_matchcol == 'county':
                county = match
            # add match, print out match
            if initial:
                print("{} -> {} in {}".format(town, match, county))
                town_index = towns[towns['town'] == town].index
                towns.loc[town_index, 'county'] = county
                towns.loc[town_index, 'new_town'] = match
            else:
                original_town = towns[towns['town2'] == town]['town'].tolist()
                if town not in printedtowns:
                    print("{} (new name: {}) -> {} in {}".format(original_town, town, match, county))
                    printedtowns.append(town)
                town_index = towns[towns['town'].apply(lambda x: x in original_town)].index
                towns.loc[town_index, 'county'] = [county] * len(town_index)
                towns.loc[town_index, 'new_town'] = [match] * len(town_index)
    return towns

In [23]:
def directTownMatch(state_cw_given, towns, col='primary_city', towncol='town'):
    print("Direct City name - county matches\n")
    # match towns directly based off crosswalk
    state_cw = state_cw_given[[col, 'county']].groupby(col).county.agg(lambda x : x.mode()[0]).reset_index()
    primary_dict = dict(zip(state_cw[col], state_cw['county']))

    if col == 'primary_city':
        towns['county'] = towns[towncol].apply(lambda x: primary_dict.get(x, np.nan))
        towns['new_town'] = [tn if not pd.isnull(cty) else np.nan for cty, tn in zip(towns['county'], towns['town'])]
    if col == 'acceptable_cities':
        for ind in towns.index:
            town = towns.loc[ind, 'town']
            county = state_cw[state_cw[col].apply(lambda x: town in x if not pd.isnull(x) else False)][
                'county'].tolist()
            if len(county) > 0:
                towns.loc[ind, 'county'] = county[0]
                towns.loc[ind, 'new_town'] = town
    t = towns[towns['county'].apply(lambda x: not pd.isnull(x))]
    if towncol == 'town':
        for tn, cty in zip(t['town'], t['county']):
            print("{} was matched to {} directly using the crosswalk".format(tn, cty))
    if towncol == 'town2':
        for tn, tn_og, cty in zip(t['town2'], t['town'], t['county']):
            print("{} (original: {}) was matched to {} directly using the crosswalk".format(tn, tn_og, cty))
    return primary_dict, towns

In [24]:
def directCountyMatch(state_cw, towns, towncol='town'):
    print("\nSome city names are actually county names")
    if towncol == 'town':
        print("Direct City (county) name - county matches\n")
    if towncol == 'town2':
        print("Direct City (county) name with string changes - county matches\n")
    # some own names are actually counties
    # match towns based off whether town name is actually county name in crosswalk
    counties = state_cw['county'].unique()
    nanindex = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
    towns.loc[nanindex, 'county'] = towns.loc[nanindex, towncol].apply(
        lambda x: x if x in counties.tolist() else np.nan)
    towns2 = towns.loc[nanindex]
    nanindex2 = towns2[towns2['county'].apply(lambda x: not pd.isnull(x))].index
    for t, c in zip(towns2.loc[nanindex2, towncol], towns2.loc[nanindex2, 'county']):
        print("{} was matched to {} using the crosswalk".format(t, c))
    return towns

In [25]:
# change column of town dataframe's type to either town or county
def addType(towns, type='town'):
    towns['name_type'] = [name_type if not pd.isnull(name_type) else type if not pd.isnull(county) else np.nan for
                          name_type, county in
                          zip(towns['name_type'], towns['county'])]
    return towns

In [26]:
city_county_cw = pd.read_excel('../data_raw/census_data/zip_code_database.xls')[['primary_city', 'acceptable_cities',
                                                                                 'unacceptable_cities', 'county',
                                                                                 'state']]

In [27]:
final_cw = pd.DataFrame(columns=['town', 'county', 'state', 'name_type'])
list_of_states = ['CT', 'GA', 'MD', 'NC', 'NH', 'NJ', 'PA', 'RI', 'SC',
                  'MA', 'VA', 'DE']

for state in list_of_states:
    print("\n{} MATCHING \n".format(state))
    # create list of towns for each state
    towns = CD_all[CD_all['state'] == state][['town']].drop_duplicates()
    towns = towns[towns['town'].apply(lambda x: not pd.isnull(x))]
    # state crosswalk
    state_cw = city_county_cw[city_county_cw['state'] == state]
    if state == 'VA':
        state_cw = city_county_cw[city_county_cw['state'].apply(lambda x: x in ['VA', 'WV'])]
    state_cw = state_cw[state_cw['county'].apply(lambda x: 'county' in x.lower() if not pd.isnull(x) else False)]

    # try direct match: town name -> crosswalk town-county
    oldtowns = towns.copy()
    primary_dict, towns = directTownMatch(state_cw, towns, col='primary_city', towncol='town')
    # label name type
    towns['name_type'] = towns['county'].apply(lambda x: 'town' if not pd.isnull(x) else np.nan)

    if state == 'CT':
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns1 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town']
        towns = fuzzyMatch(unmatched_towns1, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=True,
                           score_threshold=85)
        towns = addType(towns)

        # modify town names - towns changed names (see CT note)
        # retry fuzzy match: town name -> crosswalk town-county
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('Huntington', 'Shelton').replace('Chatham', 'East Hampton'))
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

    if state == 'GA':
        # some "town" names are actually counties
        # try direct match: town (county) name -> crosswalk county
        towns = directCountyMatch(state_cw, towns, towncol='town')
        towns = addType(towns, 'county')
    if state == 'MD':
        # remove instances where Maryland is mentioned and unabbreviate county abbreviations
        # use modified town names
        # try direct match: town (county) name -> crosswalk county
        towns['town2'] = towns['town'].apply(lambda x: x.replace('Maryland', '').replace('Co ', 'County').strip())
        towns = directCountyMatch(state_cw, towns, towncol='town2')
        towns = addType(towns, 'county')

        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='county', initial=False,
                           score_threshold=86)
        towns = addType(towns, 'county')

        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # correct a matching - Baltimore City to Baltimore County
        print("Baltimore City changed to Baltimore County")
        towns['county'] = towns['county'].apply(lambda x: x.replace('City', 'County') if not pd.isnull(x) else x)
        towns = addType(towns)
    if state == 'NC':
        # some "town" names are actually counties
        # try direct match: town (county) name -> crosswalk county
        towns = directCountyMatch(state_cw, towns, towncol='town')
        towns = addType(towns, 'county')
        # remove instances where North Carolina is mentioned and rename Tarborugh to enable matching
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('North Carolina', '').replace('Tarborugh', 'Tarboro').strip())
        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)
    if state == 'NH':
        # use acceptable_cities instead of primary_cities column to match in the crosswalk
        # try direct match: town name -> crosswalk town-county
        null_ind = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
        pdict, tn = directTownMatch(state_cw, towns.loc[null_ind], col='acceptable_cities', towncol='town')
        towns.loc[null_ind] = tn
        towns = addType(towns)
        # remove instances where New Hampshire and other geo-jurisdictional terms are used
        # rename Rockingham to enable matching
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('State', '').replace('New Hampshire', '').replace('of ', '').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Rockingham', 'Rockingham County').strip())
        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # some "town" names are actually counties
        # use modified town names
        # try direct match: town (county) name -> crosswalk county
        towns = directCountyMatch(state_cw, towns, towncol='town2')
        towns = addType(towns, 'county')
        # manual fixes for matches
        # manually adjust incorrect matches
        print("\nManual Match\n")
        for town, county in zip(['Brintwood', 'Portsmouth New Hampshire'],
                                ['Rockingham County', 'Rockingham County']):
            print("{} was matched to {}".format(town, county))
            if town == 'Brintwood':
                towns.loc[towns[towns['town'] == town].index, ['county', 'name_type']] = [county, 'town]']
            else:
                towns.loc[towns[towns['town'] == town].index, ['county', 'name_type']] = [county, 'county']
    if state == 'NJ':
        # remove instances where New Jersey is used
        towns['town2'] = towns['town'].apply(lambda x: x.replace('New Jersey', '').strip())

        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)
        # some "town" names are actually counties
        # use modified town names
        # try direct match: town (county) name -> crosswalk county
        towns = directCountyMatch(state_cw, towns, towncol='town2')
        towns = addType(towns, 'county')
    if state == 'PA':
        # some "town" names are actually counties
        # try direct match: town (county) name -> crosswalk county
        towns = directCountyMatch(state_cw, towns, towncol='town')
        towns = addType(towns, 'county')

        # use acceptable_cities instead of primary_cities column to match in the crosswalk
        # try direct match: town (county) name -> crosswalk county
        null_ind = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
        pdict, tn = directTownMatch(state_cw, towns.loc[null_ind], col='acceptable_cities', towncol='town')
        towns.loc[null_ind] = tn
        towns = addType(towns)

        # remove instances where New Jersey is used, fix some notational issues
        # correct Dauphincoy to Dauphin and categorize Tulpehocken as being in Berks County
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('Co ', 'County').replace('Delaware', 'Delaware County').strip())
        towns['town2'] = towns['town2'].apply(
            lambda x: x.replace('Pennsylvania', '').replace('County County', 'County').strip())
        towns['town2'] = towns['town2'].apply(
            lambda x: x.replace('Country', 'County').replace('Dauphincoy', 'Dauphin').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Tulpehocken', 'Berks County').strip())

        # categorize different Philadelphia neighborhoods as belonging in Philadelphia
        philreptowns = ['Blockley', 'Northan Liberties', 'Northern Liberties',
                        'The Northern Libert', 'Passyunk', 'German Town', 'Southwark',
                        'Borden Town']  # not sure on this last one...
        for town in philreptowns:
            towns['town2'] = towns['town2'].apply(lambda x: x.replace(town, 'Philadelphia'))
        towns = addType(towns)
        # use modified town names
        # try direct match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = directCountyMatch(state_cw, towns, 'town2')
        towns = addType(towns, 'county')
        # some "town" names are actually counties
        # try direct match: town (county) name -> crosswalk county
        towns = directCountyMatch(state_cw, towns, towncol='town')
        towns = addType(towns, 'county')

        # use acceptable_cities instead of primary_cities column to match in the crosswalk
        # try direct match: town (county) name -> crosswalk county
        null_ind = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
        pdict, tn = directTownMatch(state_cw, towns.loc[null_ind], col='acceptable_cities', towncol='town')
        towns.loc[null_ind] = tn
        towns = addType(towns)

        # remove instances where New Jersey is used, fix some notational issues
        # correct Dauphincoy to Dauphin and categorize Tulpehocken as being in Berks County
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('Co ', 'County').replace('Delaware', 'Delaware County').strip())
        towns['town2'] = towns['town2'].apply(
            lambda x: x.replace('Pennsylvania', '').replace('County County', 'County').strip())
        towns['town2'] = towns['town2'].apply(
            lambda x: x.replace('Country', 'County').replace('Dauphincoy', 'Dauphin').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Tulpehocken', 'Berks County').strip())

        # categorize different Philadelphia neighborhoods as belonging in Philadelphia
        philreptowns = ['Blockley', 'Northan Liberties', 'Northern Liberties',
                        'The Northern Libert', 'Passyunk', 'German Town', 'Southwark',
                        'Borden Town']  # not sure on this last one...
        for town in philreptowns:
            towns['town2'] = towns['town2'].apply(lambda x: x.replace(town, 'Philadelphia'))
        towns = addType(towns)
        # use modified town names
        # try direct match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = directCountyMatch(state_cw, towns, 'town2')
        towns = addType(towns, 'county')

        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2_1 = [x for x in towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2'] if x != '']
        towns = fuzzyMatch(unmatched_towns2_1, towns, state_cw, primary_dict, dict_matchcol='county', initial=False,
                           score_threshold=85)
        towns = addType(towns, 'county')

        # manually adjust incorrect matches
        print("\nManual Match\n")
        for town, county, s in zip(
                ['Charleston South Carolina', 'Burlington New Jersey', 'Northumberland County Virginia'],
                ['Charleston County', 'Burlington County', 'Northumberland County'],
                ['SC', 'NJ', 'VA']):
            print("{} was matched to {}".format(town, county))
            towns.loc[towns[towns['town'] == town].index, ['county', 'state']] = [county, s]
        towns = addType(towns, 'county')

        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2_1 = [x for x in towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2'] if x != '']
        towns = fuzzyMatch(unmatched_towns2_1, towns, state_cw, primary_dict, dict_matchcol='county', initial=False,
                           score_threshold=85)
        towns = addType(towns, 'county')

        # manually adjust incorrect matches
        print("\nManual Match\n")
        for town, county, s in zip(
                ['Charleston South Carolina', 'Burlington New Jersey', 'Northumberland County Virginia'],
                ['Charleston County', 'Burlington County', 'Northumberland County'],
                ['SC', 'NJ', 'VA']):
            print("{} was matched to {}".format(town, county))
            towns.loc[towns[towns['town'] == town].index, ['county', 'state']] = [county, s]
        towns = addType(towns, 'county')
    if state == 'RI':
        # remove instances where Rhode Island and other geo-jurisdictional terms are used
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('Rhode Island', '').replace('State ', '').replace('of', '').strip())
        # use modified town names
        # try fuzzy match: town name -> crosswalk town-county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # manually adjust incorrect matches
        print("\nManual Match\n")
        for town, county in zip(['Gloucester', 'Richmond'],
                                ['Providence County', 'Washington County']):
            print("{} was matched to {}".format(town, county))
            towns.loc[towns[towns['town'] == town].index, 'county'] = county
        towns = addType(towns)

    if state == 'SC':
        # remove instances where South Carolina is used, change number to character
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('South Carolina', '').replace('96', 'Ninety six').strip())

        # use modified town names
        # use acceptable_cities column
        # try fuzzy match: town (county) name -> crosswalk county
        null_ind = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
        pdict, tn = directTownMatch(state_cw, towns.loc[null_ind], col='acceptable_cities', towncol='town2')
        towns.loc[null_ind] = tn
        towns = addType(towns)

        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # some town names are actually county names
        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2_1 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2_1, towns, state_cw, primary_dict, dict_matchcol='county', initial=False,
                           score_threshold=85)
        towns = addType(towns, 'county')

        townlist = ['St. Paul\'s', 'Pee Dee', 'St. George', 'New River', 'Winyaw', 'Broad River',
                    'Toogoodoo', 'St Pauls', 'Savannah',
                    'James Island', 'St Andrews']  # last two are manual fixes
        countylist = ['Clarendon County', 'Marion County', 'Dorchester County', 'Beaufort County', 'Georgetown County',
                      'Beaufort County',
                      'Charleston County', 'Clarendon County', 'Chatham County',
                      'Charleston County', 'Richland County']
        for town, county in zip(townlist, countylist):
            print("{} was matched to {}".format(town, county))
            towns.loc[towns[towns['town'] == town].index, 'county'] = county
        towns = addType(towns)

        towns.loc[towns[towns['town'] == 'Savannah'].index, 'state'] = 'GA'

    if state == 'MA':
        # remove instances where Massachusetts, MA or State is used
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('MA', '').replace('Massachusetts', '').replace('State', '').strip())
        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

    if state == 'VA':
        # remove instances where Massachusetts, MA or State is used
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('VA', '').replace('Virginia', '').replace('Virgina', '').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('State', '').replace(' of ', '').strip())
        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns)

        # some town names are actually county names
        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2_1 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2_1, towns, state_cw, primary_dict, dict_matchcol='county', initial=False,
                           score_threshold=85)
        towns = addType(towns, 'county')

        townlist = ['Portsmouth Virginia']  # last two are manual fixes
        countylist = ['Norfolk County']
        for town, county in zip(townlist, countylist):
            print("{} was matched to {}".format(town, county))
            towns.loc[towns[towns['town'] == town].index, 'county'] = county
        towns = addType(towns)
    if state == 'DE':
        # remove instances where Massachusetts, MA or State is used
        towns['town2'] = towns['town'].apply(
            lambda x: x.replace('Delaware', '').replace('State', '').replace(' of ', '').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Kent Company', 'Kent County').strip())
        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol='primary_city', initial=False,
                           score_threshold=85)
        towns = addType(towns, 'county')

        # some town names are actually county names
        # use modified town names
        # try fuzzy match: town (county) name -> crosswalk county
        unmatched_towns2_1 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2_1, towns, state_cw, primary_dict, dict_matchcol='county', initial=False,
                           score_threshold=85)
        towns = addType(towns, 'county')

    # print out all unmatched names
    print("\nFinal Unmatched Names\n")
    t = towns[towns['county'].apply(lambda x: pd.isnull(x))]
    for tn in t['town']:
        print("{} was unable to be matched".format(tn))
    towns = towns[towns['county'].apply(lambda x: not pd.isnull(x))]
    towns['state'] = state

    # only Georgia doesn't have a town2 column
    if state not in ['GA']:
        towns.drop('town2', axis=1, inplace=True)

    # correct states for certain counties/cities in PA and SC
    if state == 'PA':
        towns.loc[towns[towns['town'] == 'Charleston South Carolina'].index, 'state'] = 'SC'
        towns.loc[towns[towns['town'] == 'Northumberland County Virginia'].index, 'state'] = 'VA'
    if state == 'SC':
        towns.loc[towns[towns['town'] == 'Savannah'].index, 'state'] = 'GA'

    final_cw = pd.concat([final_cw, towns])


CT MATCHING 

Direct City name - county matches

Hartford was matched to Hartford County directly using the crosswalk
Bolton was matched to Tolland County directly using the crosswalk
Wethersfield was matched to Hartford County directly using the crosswalk
New Haven was matched to New Haven County directly using the crosswalk
Farmington was matched to Hartford County directly using the crosswalk
New London was matched to New London County directly using the crosswalk
Cornwall was matched to Litchfield County directly using the crosswalk
Stamford was matched to Fairfield County directly using the crosswalk
East Hartford was matched to Hartford County directly using the crosswalk
Bristol was matched to Hartford County directly using the crosswalk
Lebanon was matched to New London County directly using the crosswalk
Windsor was matched to Hartford County directly using the crosswalk
Suffield was matched to Hartford County directly using the crosswalk
Berlin was matched to Hartford County

In [28]:
final_cw_all = pd.merge(CD_all[['town', 'state']].drop_duplicates(), final_cw, how = 'left', on = ['town', 'state'])
final_cw_all.reset_index(inplace=True, drop=True)

In [29]:
final_cw_all.loc[final_cw_all[final_cw_all['name_type'] != 'town'].index, 'new_town'] = np.nan

In [30]:
final_cw_all['new_state'] = final_cw_all['state']

In [31]:
# manually input county, state and name type labels
towns = ['Colchester', 'Charleston South Carolina', 'Philadelphia', 'Albany', 'Newark',
         'Northumberland County Virginia', 'Savannah', 'City of New York', 'Long Island', 'Portsmouth Virginia']
counties = ['New London County', 'Berkeley County', 'Philadelphia County', 'Albany County', 'Essex County',
            'Northumberland County', 'Chatham County', 'New York County',
            np.nan, 'Rockingham County']
states = ['CT', 'SC', 'PA', 'NY', 'NJ', 'VA', 'GA', 'NY', 'NY', 'VA']
newtowns = ['Colchester', 'Charleston', 'Philadephia', 'Albany', 'Newark',
           np.nan, 'Savannah', 'New York City', np.nan, 'Portsmouth']
for town, county, state, newtown in zip(towns, counties, states, newtowns):
    if town == 'Northumberland County Virginia':
        final_cw_all.loc[final_cw_all[final_cw_all['town'] == town].index, ['new_town', 'county', 'new_state', 'name_type']] = [newtown, county, state,
                                                                                                    'county']
    elif pd.isnull(county):
        final_cw_all.loc[final_cw_all[final_cw_all['town'] == town].index, ['new_town', 'county', 'new_state', 'name_type']] = [newtown, county, state,
                                                                                                    'other']
    else:
        final_cw_all.loc[final_cw_all[final_cw_all['town'] == town].index, ['new_town', 'county', 'new_state', 'name_type']] = [newtown, county, state,
                                                                                                    'town']

In [32]:
# manually input county, state and name type labels
colonies = ['New Hampshire', 'Massachusetts', 'Rhode Island', 'Connecticut', 'Conecticutt', 'New York', 'New Jersey',
            'Pennsylvania', 'Delaware', 'Maryland',
            'Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Vermont', 'Delawere', 'Virgina']
abbrev = ['NH', 'MA', 'RI', 'CT', 'CT', 'NY', 'NJ', 'PA', 'DE', 'MD', 'VA', 'NC', 'SC', 'GA', 'VT', 'DE', 'VA']
for colony, abrv in zip(colonies, abbrev):
    ind = final_cw_all[final_cw_all['town'] == colony].index
    final_cw_all.loc[ind, ['town', 'county', 'new_state', 'name_type', 'new_town']] = [colony, np.nan, abrv, 'state', np.nan]

In [33]:
# manual replacement of counties that changed names or boders
# manually fixing mistakes
towns = ['Gilmantown', np.nan, 'Berlington New Jersey', np.nan, np.nan, np.nan, np.nan, np.nan, '96 District',
         'Ninety six District', np.nan, np.nan, np.nan, np.nan, np.nan, 'Trenton New Jersey', 'Princeton New Jersey',
         'Newbury', 'Pembroke', 'Hopkinton', 'Salisbury', 'Canterbury', 'Concord', np.nan, np.nan, np.nan]
newtowns = ['Gilmantown', np.nan, 'Burlington', np.nan, np.nan, np.nan, np.nan, np.nan, 'Philadelphia',
         'Philadelphia', np.nan, np.nan, np.nan, np.nan, np.nan, 'Trenton', 'Princeton',
         'Newbury', 'Pembroke', 'Hopkinton', 'Salisbury', 'Canterbury', 'Concord', np.nan, np.nan, np.nan]
oldcounties = ['Belknap', 'Berkeley', 'Camden', 'Carroll', 'Columbia', 'Elk', 'Dorchester', 'Grant', 'Greenwood',
               'Greenwood', 'Hampden', 'Kershaw', 'King and Queen', 'Lycoming', 'Marion', 'Mercer', 'Mercer',
               'Merrimack', 'Merrimack', 'Merrimack', 'Merrimack', 'Merrimack', 'Merrimack', 'Norfolk', 'Perry',
               'Sullivan']
newcounties = ['Strafford', 'Charleston', 'Burlington', 'Strafford', 'Richmond', 'Northumberland', 'Charleston',
               'Hampshire', 'Abbeville',
               'Laurens', 'Hampshire', 'Lancaster', np.nan, 'Northumberland', 'Prince George\'s', 'Hunterdon',
               'Middlesex',
               'Hillsborough', 'Hillsborough', 'Hillsborough', 'Hillsborough', 'Rockingham', 'Grafton', 'Suffolk',
               'Cumberland', 'Cheshire']
for town, oldcounty, newcounty, newtown in zip(towns, oldcounties, newcounties, newtowns):
    if not pd.isnull(town):
        ind = final_cw_all[
            [t == town and oldcounty + ' County' == oc for t, oc in zip(final_cw_all['town'], final_cw_all['county'])]].index
    else:
        ind = final_cw_all[final_cw_all['county'] == oldcounty + ' County'].index
    if pd.isnull(newcounty):
        if not pd.isnull(town):
            final_cw_all.loc[ind, ['new_town', 'county']] = [newtown, newcounty]
        else:
            final_cw_all.loc[ind, ['county']] = [newcounty]
    else:
        if not pd.isnull(town):
            final_cw_all.loc[ind, ['new_town', 'county']] = [newtown, newcounty + ' County']
        else:
            final_cw_all.loc[ind, ['county']] = [newcounty + ' County']
    if len(ind) == 0:
        print(town, oldcounty)
#make corrections
ind = final_cw_all[[c == 'Charleston County' and s == 'MD' for c, s in zip(final_cw_all['county'], final_cw_all['state'])]].index
final_cw_all.loc[ind, 'county'] = 'Dorchester County'

ind = final_cw_all[
    [c == 'Prince George\'s County' and s == 'SC' for c, s in zip(final_cw_all['county'], final_cw_all['state'])]].index
final_cw_all.loc[ind, 'county'] = 'Georgetown County'

Berlington New Jersey Camden


In [34]:
# manual fixes of assignments
final_cw_all.loc[final_cw_all[final_cw_all['county'] == 'Prince George\'s County'].index, 'county'] = 'Prince Georges County'
final_cw_all.loc[final_cw_all[final_cw_all['county'] == 'Queen Anne\'s County'].index, 'county'] = 'Queen Annes County'
final_cw_all.loc[final_cw_all[final_cw_all['county'] == 'St Mary\'s County'].index, 'county'] = 'St Marys County'

final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Doden Maryland'].index, ['county', 'name_type']] = ['Anne Arundel County',
                                                                                               'county']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Huntington New Jersey'].index, ['new_town', 'county', 'name_type']] = [
    'Huntington', 'Hunterdon County', 'county']
# adding state labels
final_cw_all.loc[final_cw_all[[pd.isnull(t) and not pd.isnull(s) and s != 'FR' for t, s in
                       zip(final_cw_all['town'], final_cw_all['state'])]].index, 'name_type'] = 'state'

final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Kittery'].index, ['county', 'name_type', 'new_town']] = ['York County', 'town', 'Kittery']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Kensignton'].index, ['county', 'name_type', 'new_town']] = ['Philadelphia County',
                                                                                           'town', 'Philadelphia']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'York'].index, ['county','new_state', 'name_type','new_town']] = ['York County', 'MA', 'town', 'York']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Wells'].index, ['county', 'new_state', 'name_type','new_town']] = ['York County', 'MA', 'town', 'Wells']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'James City County Virginia'].index, ['county']] = ['James City County']

final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Cumb County Pennsylvania'].index, ['county']] = ['Cumberland County']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Cumberland'].index, 'name_type'] = 'county'
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'york town pennsylvania'].index, ['new_town', 'county', 'name_type']] = ['York Township',
    'York County', 'town']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Portsmouth New Hampshire'].index, ['name_type', 'new_town']] = [
    'town', 'Portsmouth']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Tulpehocken'].index, ['name_type', 'new_town']] = [
    'town', 'Tulpehocken']
final_cw_all.loc[final_cw_all.query('town == "Long Cames" and name_type.isnull()').index, ['name_type', 'county']] = ['county', 'Abbeville County']
final_cw_all.loc[final_cw_all.query('town == "Springfield"').index, ['name_type', 'new_town', 'county', 'new_state']] = ['town', 'Springfield', 'Hampshire County', 'MA']

# Manual fixes of name_type
final_cw_all.loc[final_cw_all['town'] == 'Virginia and Philadelphia', 'name_type'] = 'state'

# some chester counties are mislabelled
chester_ind = final_cw_all[
    final_cw_all['town'].apply(lambda x: 'Chester' in x and 'Massachusetts' not in x if not pd.isnull(x) else False)].index
final_cw_all.loc[chester_ind, ['county', 'name_type', 'new_town']] = ['Chester County', 'county', np.nan]

# correctly label some counties
county_ind = final_cw_all[[nt == 'town' and (
        'County of Philadelphia' in c or 'Paxton Tot Dauphin County' in c or 'Somerset County New Jersey' in c)
                       for nt, c in zip(final_cw_all['name_type'], final_cw_all['town'])]].index
final_cw_all.loc[county_ind, 'name_type'] = 'county'

In [35]:
final_cw_all.loc[final_cw_all[final_cw_all['town'].apply(lambda x: "State" in x if not pd.isnull(x) else False)].query('name_type.isnull()').index, ['name_type']] = ['state']
# assigning unassinable towns as states
final_cw_all.loc[final_cw_all.query('town == "North Hampshire" and name_type.isnull()').index, ['name_type']] = ['state']
final_cw_all.loc[final_cw_all.query('town == "Long Island" and name_type.isnull()').index, ['name_type']] = ['state']
final_cw_all.loc[final_cw_all.query('town == "Carolina" and name_type.isnull()').index, ['name_type']] = ['state']
final_cw_all.loc[final_cw_all.query('town == "Doden" and name_type.isnull()').index, ['name_type']] = ['state']
final_cw_all.loc[final_cw_all.query('town == "Isaac" and name_type.isnull()').index, ['name_type']] = ['state']
final_cw_all.loc[final_cw_all.query('town == "Glouster" and name_type.isnull()').index, ['name_type']] = ['state']
final_cw_all.loc[final_cw_all.query('town == "W Callisters" and name_type.isnull()').index, ['name_type']] = ['state']

In [36]:
final_cw_all.loc[final_cw_all[final_cw_all['new_town'].isnull()].query('name_type == "town"').index, 'new_town'] = final_cw_all[final_cw_all['new_town'].isnull()].query('name_type == "town"')['town']

In [37]:
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Richard Sennings'].index, ['new_town', 'county', 'new_state', 'name_type']] = [np.nan, np.nan, 'SC', 'state']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'W Callisters Town'].index, ['new_town', 'county', 'new_state', 'name_type']] = [np.nan, np.nan, 'PA', 'state']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Connecticutt'].index, ['new_town', 'county', 'new_state', 'name_type']] = [np.nan, np.nan, 'CT', 'state']

In [38]:
final_cw_all['country'] = np.nan

In [39]:
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Rotterdam'].index, ['new_state','country', 'name_type']] = [np.nan, 'NL', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'St Eustalia'].index, ['new_state','country', 'name_type']] = [np.nan, 'NL', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'London'].index, ['new_state','country',  'name_type', 'new_town']] = [np.nan, 'GB', 'country',
                                                                                                                             np.nan]
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Eustatia'].index, ['new_state','country', 'name_type']] = [np.nan, 'VI', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Barbadoes'].index, ['new_state','country', 'name_type']] = [np.nan, 'BB', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Sta Croix'].index, ['new_state','country', 'name_type']] = [np.nan, 'VI', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Maidenhead'].index, ['new_state','country', 'name_type']] = [np.nan, 'GB', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Bermuda'].index, ['new_state','country', 'name_type']] = [np.nan, 'BM', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['state'] == 'FR'].index, ['new_state','country', 'name_type']] = [np.nan, 'FR', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'St. Thomas'].index, ['new_state','country', 'name_type']] = [np.nan, 'VI', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'St Thomas'].index, ['new_state','country', 'name_type']] = [np.nan, 'VI', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Edinburgh'].index, ['new_state','country', 'name_type']] = [np.nan, 'GB', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Great Britain'].index, ['new_state','country', 'name_type']] = [np.nan, 'GB', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Bourdeaux'].index, ['new_state','country', 'name_type']] = [np.nan, 'FR', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'The River May'].index, ['new_state','country', 'name_type']] = [np.nan, 'VI', 'country']
final_cw_all.loc[final_cw_all[final_cw_all['town'] == 'Register of the Treasury'].index, ['new_state','country', 'name_type']] = [np.nan, 'US', 'country']

In [40]:
final_cw_all.loc[final_cw_all[final_cw_all['country'].isnull()].index, 'country'] = 'US'

In [41]:
final_cw_all.query(
    'name_type == "town]"')

,town,state,county,name_type,new_town,new_state,country
256,Brintwood,NH,Rockingham County,town],NaN,NH,US


In [42]:
# fix state
ind = final_cw_all.query(
    'name_type == "other" and not state.isna() and state != "BVI" and state != "BM" and state != "GB" and state != "VI" and state != "FR" and state != "US"').index
final_cw_all.loc[ind, 'name_type'] = 'state'
final_cw_all.loc[final_cw_all[final_cw_all['name_type'] == 'town]'].index, 'name_type'] = 'town'

In [43]:
# drop duplicates that occur for some reason
final_cw_all.drop_duplicates(inplace=True)
# add labels for county and name type to CD_all
CD_all = pd.merge(CD_all, final_cw_all, on=['town', 'state'], how='left')

In [44]:
CD_all['6p_total'] = CD_all['6p_Dollar'] + CD_all['6p_Cents'] / 100

In [ ]:
CD_all.to_csv('../data_clean/aggregated_CD_post1790.csv')

In [45]:
grouped_assets = CD_all.groupby(['county', 'state']).agg({'6p_total': ['sum', 'count', 'mean']})
grouped_assets.columns = grouped_assets.columns.map('_'.join).str.strip('_')
grouped_assets.columns = ['6p_total', 'debtholder_county_count', 'mean_6p_held']
grouped_assets = grouped_assets.reset_index()

In [47]:
countyPop = pd.read_csv('../data_raw/census_data/countyPopulation.csv').drop(0).reset_index(drop=True)
sub_cols = ['Area Name', 'State/US Abbreviation', 'Total Population', 'White Male', 'White Male Age 16 Years and over']
county_subset = countyPop[sub_cols]
merged_geography = pd.merge(county_subset, grouped_assets, left_on=['Area Name', 'State/US Abbreviation'],
                            right_on=['county', 'state'], how='right')
final_asset_data = merged_geography.drop(['Area Name', 'State/US Abbreviation'], axis=1)
final_asset_data['debt_per_capita'] = final_asset_data['6p_total'] / final_asset_data['Total Population'].apply(
    lambda x: float(x))
final_asset_data['debt_per_white_male'] = final_asset_data['6p_total'] / final_asset_data['White Male'].apply(
    lambda x: float(x))

In [ ]:
# for david to use when doing histograms of occupations vs no occupations and grouping occupations
# for Jiacheng to use when making table
CD_all[['town', 'state', 'county', 'name_type']].drop_duplicates().to_csv('../../Data/AssetGeography/county_cw.csv')
# for Maria to use when making maps
final_asset_data.to_csv('../../Data/AssetGeography/county_debt_total.csv')

In [ ]:
print(CD_all[CD_all['county'].apply(lambda x: not pd.isnull(x))]['6p_Dollar'].sum() / CD_all['6p_Dollar'].sum())
print(CD_all[CD_all['county'].apply(lambda x: not pd.isnull(x))].shape[0] / CD_all.shape[0])